In [1]:
import numpy as np 
import pandas as pd

# pandas Time zone information

In [2]:
rng = pd.date_range('3/6/2016 00:00', periods = 15, freq = 'd')
rng.tz

In [3]:
rng = pd.date_range('3/6/2016 00:00', periods = 15, freq = 'd', tz = 'Europe/London')
rng.tz

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

### Getting lists of time zones

In [4]:
from pytz import common_timezones, all_timezones
print(len(common_timezones))

439


In [5]:
print(len(all_timezones))

593


### What are some time zones not considered 'common'?

In [6]:
print(set(all_timezones) - set(common_timezones))

{'Pacific/Yap', 'US/Michigan', 'Etc/UTC', 'HST', 'Etc/GMT+7', 'NZ', 'Etc/GMT+12', 'Australia/Yancowinna', 'Australia/Canberra', 'Iran', 'WET', 'America/Catamarca', 'GMT+0', 'Singapore', 'Atlantic/Faeroe', 'Europe/Tiraspol', 'America/Santa_Isabel', 'Kwajalein', 'America/Cordoba', 'Brazil/East', 'Asia/Ashkhabad', 'GB-Eire', 'Mexico/BajaSur', 'Asia/Macao', 'Eire', 'Pacific/Truk', 'CST6CDT', 'Israel', 'Asia/Kashgar', 'Etc/Greenwich', 'America/Knox_IN', 'America/Jujuy', 'Africa/Timbuktu', 'W-SU', 'America/Shiprock', 'Etc/GMT+4', 'Etc/GMT+3', 'Etc/GMT+6', 'Asia/Ujung_Pandang', 'Etc/GMT+5', 'Egypt', 'America/Coral_Harbour', 'Etc/GMT-6', 'Brazil/West', 'Jamaica', 'Australia/West', 'MET', 'Zulu', 'Etc/GMT-8', 'Etc/GMT-5', 'Etc/GMT-10', 'Etc/Universal', 'Etc/GMT-14', 'Etc/GMT', 'PRC', 'Etc/GMT+9', 'PST8PDT', 'Asia/Tel_Aviv', 'MST', 'EST', 'GMT0', 'Australia/LHI', 'Asia/Calcutta', 'America/Argentina/ComodRivadavia', 'America/Porto_Acre', 'US/Aleutian', 'Poland', 'GB', 'Asia/Chungking', 'Chile/Con

### Localizing a timestamp

In [7]:
t_naive = pd.Timestamp('2016-05-05 08:15')
t_naive

Timestamp('2016-05-05 08:15:00')

In [8]:
t = t_naive.tz_localize(tz = 'US/Central')
t

Timestamp('2016-05-05 08:15:00-0500', tz='US/Central')

In [9]:
t.tz_convert('Asia/Tokyo')

Timestamp('2016-05-05 22:15:00+0900', tz='Asia/Tokyo')

### What is the difference between tz_convert and tz_localize?

hint: try to run tz_convert on a naive time stamp

_Reply:_ `tz_localize` is used to add a local time zone while `tz_convert` changes the time zone. The video goes through some examples of error reports to show how you can only run `tz_convert` after `tz_localize`, or some equivalent, has been run.

### Ambiguous times

In [10]:
# You'll get weirdness with timezones based on daylight savings:
rng = pd.date_range('2016-03-10', periods = 10, tz = 'US/Eastern')
ts = pd.Series(range(10), index = rng)
# What do you notice below?
ts

2016-03-10 00:00:00-05:00    0
2016-03-11 00:00:00-05:00    1
2016-03-12 00:00:00-05:00    2
2016-03-13 00:00:00-05:00    3
2016-03-14 00:00:00-04:00    4
2016-03-15 00:00:00-04:00    5
2016-03-16 00:00:00-04:00    6
2016-03-17 00:00:00-04:00    7
2016-03-18 00:00:00-04:00    8
2016-03-19 00:00:00-04:00    9
Freq: D, dtype: int32

In [11]:
# For the same reason you can run into 'ambiguous' dates
rng_hourly = pd.DatetimeIndex(['11/06/2011 00:00', '11/06/2011 01:00','11/06/2011 01:00', '11/06/2011 02:00','11/06/2011 03:00'])

In [12]:
# What happens if we localize?
rng_hourly.tz_localize('US/Central', ambiguous = 'infer')

DatetimeIndex(['2011-11-06 00:00:00-05:00', '2011-11-06 01:00:00-05:00',
               '2011-11-06 01:00:00-06:00', '2011-11-06 02:00:00-06:00',
               '2011-11-06 03:00:00-06:00'],
              dtype='datetime64[ns, US/Central]', freq=None)

### How do we deal with this ambiguous time error?
hint: http://pandas.pydata.org/pandas-docs/stable/timeseries.html#ambiguous-times-when-localizing

In [13]:
# %load snippets/ambig.py
rng_hourly.tz_localize('US/Central', ambiguous = 'infer')

DatetimeIndex(['2011-11-06 00:00:00-05:00', '2011-11-06 01:00:00-05:00',
               '2011-11-06 01:00:00-06:00', '2011-11-06 02:00:00-06:00',
               '2011-11-06 03:00:00-06:00'],
              dtype='datetime64[ns, US/Central]', freq=None)

### How can we check whether the inference did what we wanted?

In [14]:
# %load snippets/confirm.py
rng_hourly.tz_localize('US/Central', ambiguous = 'infer').tz_convert('utc')

DatetimeIndex(['2011-11-06 05:00:00+00:00', '2011-11-06 06:00:00+00:00',
               '2011-11-06 07:00:00+00:00', '2011-11-06 08:00:00+00:00',
               '2011-11-06 09:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### Pandas goes to amazing length to try to figure things out for you

In [15]:
# What's going on here?
pd.Timestamp('2016-03-13 02:00', tz = 'US/Eastern')

NonExistentTimeError: 2016-03-13 02:00:00

This time doesn't exist and forcing pandas to try and use it generates an error. The time doesn't exist as this is the hour that is lost when clocks are set forward in the spring to start daylight savings time.